In [0]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential 
from keras.layers import Input, Masking
from keras.layers import Dense, Input, Reshape
from keras.layers import Dropout, Activation
from keras.layers import Dense, GlobalAveragePooling1D, Activation
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM, SimpleRNN
from keras.layers import Embedding
import keras.backend as K 
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers.merge import concatenate
from keras.layers.convolutional import MaxPooling1D
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.phrases import Phraser, Phrases

Using TensorFlow backend.


In [0]:
%%time
df = pd.read_excel('data.xlsx')
df = df[df['attribute_name'] == 'style'].drop(columns = ['attribute_name'])

df['attribute_value'] = df['attribute_value'].apply(lambda x: 'semifit' if x == 'semi fit' else x)
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'fittedtailor' if x == 'fitted tailor' else x)
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'semifit' if x == 'semifitte' else x)
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'fittedtailor' if x == 'fittedtailore' else x)
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'straightregular' if x == 'straight regular' else x)

CPU times: user 7.94 s, sys: 66.9 ms, total: 8.01 s
Wall time: 8.03 s


In [0]:
df.head()

,product_id,brand,product_full_name,description,brand_category,attribute_value
0,01DPGV4YRP3Z8J85DASGZ1Y99W,frame,les second medium noir,"minimal , modern styling meet refined luxury l...",accessory,casual
5,01DPH1DEN9G2WM7WAMJMD0A9W4,j crew,tie waist shirtdress stripe,take classic button silhouette turn ultra flat...,dressesandjumpsuits,casual
12,01E2KYW52BAG606GQ7A9H5R0KD,alo,interval microfleece pullover hoodie,articulate seam extra wide rib hem create shap...,unknown,casual
13,01DT513RRYT3SKH6X25G5VCH6B,chlo,leather ankle boot,heel measure approximately 55 mm 2 inch 30 mm ...,shoe boots ankle,androgynous
20,01E2KM0KW6NB1JKMZVRXR6H8G2,alo,stadium quarter zip hoodie,supersoft hoodie design elastic hem cuff perfe...,unknown,casual


In [0]:
df['text'] = (df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category']).apply(str)

In [0]:
X = df['text'].values
y = pd.get_dummies(df['attribute_value'])
label_list = y.columns
y = y.values

In [0]:
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(X)

In [0]:
X = tokenizer.texts_to_sequences(X)

In [0]:
length = max([len(s.split()) for s in df['text']])
vocab_size = len(tokenizer.word_index) + 1

In [0]:
X = pad_sequences(X, maxlen=length, padding='post')

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [0]:
resampler = SMOTETomek(sampling_strategy = 'auto')
X_train, y_train = resampler.fit_resample(X_train, y_train)

In [0]:
num_classes = y.shape[1]

In [0]:
def load_glove_vectors():
    embeddings_index = {}
    with open('glove.6B.100d.txt') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index


embeddings_index = load_glove_vectors()

Loaded 400000 word vectors.


In [0]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: # check that it is an actual word that we have embeddings for
        embedding_matrix[i] = embedding_vector

In [0]:
def define_model():
    
    model = Sequential()
    model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=length, trainable=False))
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.15, activation = 'relu'))
    model.add(LSTM(64, return_sequences=False, dropout=0.2, recurrent_dropout=0.15, activation = 'relu'))
    #model.add(Flatten())
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(num_classes, activation='sigmoid'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

def define_model():
    
    model = Sequential()
    model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=length, trainable=False))
    #model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.15, activation = 'relu')
    LSTM(128, return_sequences=False, dropout=0.2, recurrent_dropout=0.15, activation = 'relu')
    model.add(Flatten())
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(num_classes, activation='sigmoid'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [0]:
model = define_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 155, 100)          611400    
_________________________________________________________________
masking_1 (Masking)          (None, 155, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 155, 64)           42240     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_2 (Dense)              (None, 12)                204       
Total params: 687,908
Trainable params: 76,508
Non-trainable params: 611,400
___________________________________________

In [0]:
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=512)

Train on 21980 samples, validate on 5496 samples
Epoch 1/10
21980/21980 [==============================] - 49s 2ms/step - loss: 0.6280 - accuracy: 0.7963 - val_loss: 0.5437 - val_accuracy: 0.8409
Epoch 2/10
21980/21980 [==============================] - 50s 2ms/step - loss: 0.4775 - accuracy: 0.8650 - val_loss: 0.4806 - val_accuracy: 0.8589
Epoch 3/10
21980/21980 [==============================] - 66s 3ms/step - loss: 0.4131 - accuracy: 0.9020 - val_loss: 0.4330 - val_accuracy: 0.9167
Epoch 4/10
21980/21980 [==============================] - 58s 3ms/step - loss: 0.3702 - accuracy: 0.9166 - val_loss: 0.4052 - val_accuracy: 0.9167
Epoch 5/10
21980/21980 [==============================] - 49s 2ms/step - loss: 0.3415 - accuracy: 0.9167 - val_loss: 0.3900 - val_accuracy: 0.9167
Epoch 6/10
21980/21980 [==============================] - 54s 2ms/step - loss: 0.3416 - accuracy: 0.9167 - val_loss: 0.4042 - val_accuracy: 0.9167
Epoch 7/10
21980/21980 [==============================] - 51s 2ms/ste

In [0]:
model.evaluate(X_test, y_test)

1089/1089 [==============================] - 1s 908us/step


[0.3072939977884512, 0.9166664481163025]

In [0]:
results = model.predict(X)

In [0]:
results_df = pd.DataFrame()
results_mask = results > 0.1
for i in range(len(label_list)):
    results_df[label_list[i]] = results_mask[:,i]
    results_df[label_list[i]] = results_df[label_list[i]].apply(int)

In [0]:
results_df['sum']  = 0
for key in label_list:
    results_df['sum'] = results_df['sum'] + results_df[key]
(results_df['sum'] == 0).sum()

0

In [0]:
results_df = results_df.drop(columns = ['sum'])
for key in label_list:
    results_df[key] = results_df[key].apply(lambda x: key if x == 1 else '')

results_df['attribute_value'] = ''
for key in label_list:
    results_df['attribute_value'] = results_df['attribute_value'] + ' ' + results_df[key]

results_df['attribute_value'] = results_df['attribute_value'].apply(lambda x: ', '.join(x.split()))
results_df.head()

,androgynous,athleisure,boho,business casual,businesscasual,casual,classic,edgy,glam,modern,retro,romantic,attribute_value
0,androgynous,athleisure,boho,business casual,businesscasual,casual,classic,edgy,glam,modern,,,"androgynous, athleisure, boho, business, casua..."
1,androgynous,athleisure,boho,business casual,businesscasual,casual,classic,edgy,,,,,"androgynous, athleisure, boho, business, casua..."
2,androgynous,athleisure,boho,business casual,businesscasual,casual,classic,edgy,,modern,,,"androgynous, athleisure, boho, business, casua..."
3,androgynous,athleisure,boho,business casual,businesscasual,casual,classic,edgy,,,,,"androgynous, athleisure, boho, business, casua..."
4,androgynous,athleisure,boho,business casual,businesscasual,casual,classic,edgy,,modern,,,"androgynous, athleisure, boho, business, casua..."


In [0]:
results_df['attribute_value'].value_counts()

androgynous, athleisure, boho, business, casual, businesscasual, casual, classic, edgy                                   4876
androgynous, athleisure, boho, business, casual, businesscasual, casual, classic, edgy, modern                           1972
androgynous, athleisure, boho, business, casual, businesscasual, casual, classic, edgy, glam, modern, retro, romantic    1384
androgynous, athleisure, boho, business, casual, businesscasual, casual, classic, edgy, glam, modern                     1307
androgynous, athleisure, boho, business, casual, casual, classic, edgy                                                    408
athleisure, classic                                                                                                       223
athleisure, boho, business, casual, casual, classic, edgy                                                                 138
athleisure, boho, business, casual, classic                                                                           

In [0]:
import re 
from gensim.parsing.preprocessing import remove_stopwords
import spacy
import pickle
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('model.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
def get_fit(brand, product_full_name, description, details, brand_category):
    def clean_text(x):
        try:
            x = re.sub(r'<.*?>', '',x)
            x = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", x)
            x = re.sub(r"\'s", " \'s", x)
            x = re.sub(r"\'ve", " \'ve", x)
            x = re.sub(r"n\'t", " n\'t", x)
            x = re.sub(r"\'re", " \'re", x)
            x = re.sub(r"\'d", " \'d", x)
            x = re.sub(r"\'ll", " \'ll", x)
            x = re.sub(r",", " , ", x)
            x = re.sub(r"!", " ! ", x)
            x = re.sub(r"\(", "", x)
            x = re.sub(r"\)", "", x)
            x = re.sub(r"\?", "", x)
            x = re.sub(r"/", "", x)
            x = re.sub(r"\s{2,}", " ", x)
            return x.lower()
        except:
            return ''

    def lemmatizer(x):
        return ' '.join([token.lemma_ for token in nlp(x)])
        
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    with open('model.pickle', 'rb') as handle:
        model = pickle.load(handle)

    data = pd.DataFrame({'brand':brand,'product_full_name':product_full_name,'description':description,'details':details,'brand_category':brand_category},index=[0]) 
    df = data.copy()
    for key in df.columns:
        df[key] = df[key].apply(clean_text)
        df[key] = df[key].apply(remove_stopwords)
        df[key] = df[key].apply(lemmatizer)
        df[key] = df[key].apply(clean_text)
        df[key] = df[key].apply(remove_stopwords)
    df['text'] = (df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category'] + ' ' + df['details']).apply(str)
    X = df['text'].values
    X = tokenizer.texts_to_sequences(X)
    X = pad_sequences(X, maxlen=length, padding='post')
    results = model.predict(X)
    results_df = pd.DataFrame()
    results_mask = results > 0.1
    for i in range(len(label_list)):
        results_df[label_list[i]] = results_mask[:,i]
        results_df[label_list[i]] = results_df[label_list[i]].apply(int)
    results_df['sum']  = 0
    for key in label_list:
        results_df['sum'] = results_df['sum'] + results_df[key]
    (results_df['sum'] == 0).sum()
    results_df = results_df.drop(columns = ['sum'])
    for key in label_list:
        results_df[key] = results_df[key].apply(lambda x: key if x == 1 else '')

    results_df['attribute_value'] = ''
    for key in label_list:
        results_df['attribute_value'] = results_df['attribute_value'] + ' ' + results_df[key]

    results_df['attribute_value'] = results_df['attribute_value'].apply(lambda x: ', '.join(x.split()))
    data['attribute_value'] = results_df['attribute_value']
    return data

In [0]:
brand = "frame"
product_full_name = "les second medium noir"
description = "'minimal , modern styling meet refined luxury les second caba tote craft exquisite leather , structured handbag equip adjustable high polish peg buttonhole double drop handle , center welt seam , detachable pouch , frame logo discreetly emboss'"
details = np.nan
brand_category = 'accessory'

In [0]:
get_fit(brand, product_full_name, description, details, brand_category)

,brand,product_full_name,description,details,brand_category,attribute_value
0,frame,les second medium noir,"'minimal , modern styling meet refined luxury ...",NaN,accessory,"androgynous, athleisure, boho, business, casua..."
